In [43]:
import pandas as pd
from requests import get
from bs4 import BeautifulSoup
from basketball_reference_scraper.utils import get_game_suffix, get_player_suffix

name = "Stephen Curry"
suffix = get_player_suffix(name).replace(".html","")

In [44]:
suffix

'/players/c/curryst01'

In [45]:
season_year = "2018"

In [50]:
url = "https://www.basketball-reference.com" + suffix+ "/shooting/"+season_year

In [51]:
url

'https://www.basketball-reference.com/players/c/curryst01/shooting/2018'

In [53]:
r = get(f'https://www.basketball-reference.com{suffix}/shooting/{season_year}')

In [139]:
print(url)
r = get(url)
soup = BeautifulSoup(r.content)

https://www.basketball-reference.com/players/c/curryst01/shooting/2018


In [167]:
r.content

b'\n<!DOCTYPE html>\n<html data-version="klecko-" data-root="/home/bbr/www" itemscope itemtype="https://schema.org/WebSite" lang="en" class="no-js" >\n<head>\n    <meta charset="utf-8">\n    <meta http-equiv="x-ua-compatible" content="ie=edge">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=2.0" />\n    <link rel="dns-prefetch" href="https://d2p3bygnnzw9w3.cloudfront.net/req/202006191" />\n\n<!-- no:cookie fast load the css.           -->\n<script>function gup(n) {n = n.replace(/[\\[]/, \'\\\\[\').replace(/[\\]]/, \'\\\\]\'); var r = new RegExp(\'[\\\\?&]\'+n+\'=([^&#]*)\'); var re = r.exec(location.search);   return re === null?\'\':decodeURIComponent(re[1].replace(/\\+/g,\' \'));}; document.srdev = gup(\'srdev\')</script>\n<link rel="preconnect" href="https://d2p3bygnnzw9w3.cloudfront.net" crossorigin>\n<link rel="preconnect" href="https://d2cwpp38twqe55.cloudfront.net" crossorigin>\n <style>html,body{margin:0;padding:0;font:14px/1.25 "Helvet

In [162]:
soup.findAll("div", attrs={'class': f'table_outer_container'})[1]
# soup.findAll("div", class_ = "table_wrapper")[1].find("div", class_="table_wrapper")


# ls

IndexError: list index out of range

In [ ]:
def get_player_shot_chart(name, season_year):
    
    suffix = get_player_suffix(name)
    r = get(f'https://www.basketball-reference.com/{suffix}/shooting/{season_year}')
    if r.status_code==200:
        soup = BeautifulSoup(r.content, 'html.parser')
        shot_chart_div = soup.find('div', attrs={'id': f'shots-{team1}'})

        df1 = pd.DataFrame()
        for div in shot_chart1_div.find_all('div'):
            if 'style' not in div.attrs or 'tip' not in div.attrs:
                continue
            location = get_location(div.attrs['style'])
            description = get_description(div.attrs['tip'])
            shot_d = {**location, **description}
            shot_df = pd.DataFrame.from_dict([shot_d])
            df1 = df1.append(shot_df)
        df1 = df1.reset_index()
        df1 = df1.drop('index', axis=1)
        df2 = pd.DataFrame()
        for div in shot_chart2_div.find_all('div'):
            if 'style' not in div.attrs or 'tip' not in div.attrs:
                continue
            location = get_location(div.attrs['style'])
            description = get_description(div.attrs['tip'])
            shot_d = {**location, **description}
            shot_df = pd.DataFrame.from_dict([shot_d])
            df2 = df2.append(shot_df)
        df2 = df2.reset_index()
        df2 = df2.drop('index', axis=1)

        return {f'{team1}': df1, f'{team2}': df2}

In [1]:
from basketball_reference_scraper.shot_charts import get_shot_chart

s = get_shot_chart('2020-01-13', 'CHI', 'BOS')
print(s)

{'CHI':           x        y  QUARTER TIME_REMAINING              PLAYER MAKE_MISS  \
0   13.8 ft  26.7 ft        1        11:17.0     Lauri Markkanen      MISS   
1   31.7 ft  17.6 ft        1        10:37.0    Tomáš Satoranský      MISS   
2   32.0 ft  8.26 ft        1        10:06.0     Lauri Markkanen      MISS   
3   24.0 ft  2.68 ft        1        10:05.0     Lauri Markkanen      MAKE   
4   24.5 ft  4.97 ft        1         9:13.0      Daniel Gafford      MAKE   
..      ...      ...      ...            ...                 ...       ...   
76  25.3 ft  3.58 ft        4         3:35.0         Zach LaVine      MAKE   
77  8.20 ft  27.8 ft        4         2:54.0         Zach LaVine      MAKE   
78  17.7 ft  11.2 ft        4         1:47.0  Shaquille Harrison      MISS   
79  17.2 ft  16.4 ft        4         0:54.0    Denzel Valentine      MAKE   
80   6.1 ft  20.7 ft        4         0:18.0    Denzel Valentine      MAKE   

    VALUE DISTANCE  
0       3    26 ft  
1       2    

In [4]:
import requests
import json
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from matplotlib.patches import Circle, Rectangle, Arc

import streamlit as st

# Set up player dict
player_dict = {}
player_dict["James Harden"] = 201935
player_dict["Lebron James"] = 2544
player_dict["Trae Young"] = 1629027
player_dict["Steph Curry"] = 201939
player_dict["Kawhi Leonard"] = 202695

# URL for requests
headers = {
		'Host': 'stats.nba.com',
		'Connection': 'keep-alive',
		'Accept': 'application/json, text/plain, */*',
		'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
		'Referer': 'https://stats.nba.com/',
		"x-nba-stats-origin": "stats",
		"x-nba-stats-token": "true",
		'Accept-Encoding': 'gzip, deflate, br',
		'Accept-Language': 'en-US,en;q=0.9',
	}

url_base = 'https://stats.nba.com/stats/shotchartdetail'


player_id = 201935
season_type = "Regular Season"

parameters = {
	'ContextMeasure': 'FGA',
	'LastNGames': 0,
	'LeagueID': '00',
	'Month': 0,
	'OpponentTeamID': 0,
	'Period': 0,
	'PlayerID': player_id,
	'SeasonType': season_type,
	'TeamID': 0,
	'VsDivision': '',
	'VsConference': '',
	'SeasonSegment': '',
	'Season': '2018-19',
	'RookieYear': '',
	'PlayerPosition': '',
	'Outcome': '',
	'Location': '',
	'GameSegment': '',
	'GameId': '',
	'DateTo': '',
	'DateFrom': ''
}

response = requests.get(url_base, params=parameters, headers=headers)

content = json.loads(response.content)


In [6]:

headers = content["resultSets"][0]["headers"]
shots = content["resultSets"][0]["rowSet"]

In [7]:
headers

['GRID_TYPE',
 'GAME_ID',
 'GAME_EVENT_ID',
 'PLAYER_ID',
 'PLAYER_NAME',
 'TEAM_ID',
 'TEAM_NAME',
 'PERIOD',
 'MINUTES_REMAINING',
 'SECONDS_REMAINING',
 'EVENT_TYPE',
 'ACTION_TYPE',
 'SHOT_TYPE',
 'SHOT_ZONE_BASIC',
 'SHOT_ZONE_AREA',
 'SHOT_ZONE_RANGE',
 'SHOT_DISTANCE',
 'LOC_X',
 'LOC_Y',
 'SHOT_ATTEMPTED_FLAG',
 'SHOT_MADE_FLAG',
 'GAME_DATE',
 'HTM',
 'VTM']

In [8]:
shots

[['Shot Chart Detail',
  '0021800009',
  11,
  201935,
  'James Harden',
  1610612745,
  'Houston Rockets',
  1,
  11,
  17,
  'Missed Shot',
  'Driving Finger Roll Layup Shot',
  '2PT Field Goal',
  'Restricted Area',
  'Center(C)',
  'Less Than 8 ft.',
  1,
  3,
  12,
  1,
  0,
  '20181017',
  'HOU',
  'NOP'],
 ['Shot Chart Detail',
  '0021800009',
  28,
  201935,
  'James Harden',
  1610612745,
  'Houston Rockets',
  1,
  9,
  33,
  'Missed Shot',
  'Step Back Jump shot',
  '3PT Field Goal',
  'Above the Break 3',
  'Center(C)',
  '24+ ft.',
  27,
  -61,
  264,
  1,
  0,
  '20181017',
  'HOU',
  'NOP'],
 ['Shot Chart Detail',
  '0021800009',
  55,
  201935,
  'James Harden',
  1610612745,
  'Houston Rockets',
  1,
  7,
  51,
  'Missed Shot',
  'Step Back Jump shot',
  '3PT Field Goal',
  'Above the Break 3',
  'Center(C)',
  '24+ ft.',
  25,
  -1,
  259,
  1,
  0,
  '20181017',
  'HOU',
  'NOP'],
 ['Shot Chart Detail',
  '0021800009',
  103,
  201935,
  'James Harden',
  1610612745,